In [2]:
"""

This file will generates some preprocessed data accroding to the existing studies:
'chao_cfnn', 'li_svm', 'duan_rsfc', 'wang_alex', 'yu_gnn', 'zhu_xgboost'

"""

import numpy as np 
import sys 
import os
import sys
from scipy import stats

sys.path.append('/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning')

from utils.fnirs_utils import avg_every_ten_point_in_last_dimension
from utils.fnirs_utils import get_chao_cfnn_novel_feature
from utils.fnirs_utils import li_svm_compute_10_fetures
from utils.fnirs_utils import get_duan_rsfc_data
from utils.fnirs_utils import wang_alex_feature_selection
from utils.fnirs_utils import temporal_feature_extract_yu_gnn_full
from utils.fnirs_utils import get_10_features_xgboost_zhu

# load data 
def get_avg_hbo_hbr():
    data_fold = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/allData/prognosis/pre_treatment_hamd_reduction_50/'
    data = np.load(data_fold + 'hb_data.npy')

    hbo = data[..., 0:1250].copy()
    hbr = data[..., 1252:].copy()

    hbo = avg_every_ten_point_in_last_dimension(hbo)
    hbr = avg_every_ten_point_in_last_dimension(hbr)

    return hbo, hbr
data_fold = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/allData/prognosis/pre_treatment_hamd_reduction_50/'

label = data_fold + 'label.npy'
label = np.load(label)


2024-03-23 16:34:34.967457: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-23 16:34:34.988058: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:7704] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-23 16:34:34.988078: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-23 16:34:34.988087: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1520] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-23 16:34:34.993050: I tensorflow/core/platform/cpu_feature_g

In [3]:
all_model_name = ['chao_cfnn', 'li_svm', 'duan_rsfc', 'wang_alex', 'yu_gnn', 'zhu_xgboost']
MAIN_FOLD = '/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/allData/prognosis/pretreatment_benchmarks/' 
for model_name in all_model_name:
    model_save_fold = MAIN_FOLD + model_name + '/'
    if not os.path.exists(model_save_fold):
        os.makedirs(model_save_fold)
    
    hbo, hbr = get_avg_hbo_hbr()
    input = np.concatenate((hbo[..., np.newaxis], hbr[..., np.newaxis]), axis=-1)

    if model_name == 'chao_cfnn':
        feature = get_chao_cfnn_novel_feature(hbo, hbr)
    if model_name == 'li_svm':
        feature = li_svm_compute_10_fetures(hbo)
        feature = feature.reshape(feature.shape[0], -1)   
    if model_name == 'duan_rsfc':
        feature = get_duan_rsfc_data(hbo)
    if model_name == 'wang_alex':
        feature = wang_alex_feature_selection(hbo, index_task_start=10,index_task_end=70,fs=1)
    if model_name == 'yu_gnn':
        feature = temporal_feature_extract_yu_gnn_full(input, index_start=10, index_end=70, hbo_type=0, hbr_type=1)
    if model_name == 'zhu_xgboost':
        feature = get_10_features_xgboost_zhu(hbo, index_start=10, task_duration=60)
        feature = feature.reshape(feature.shape[0], -1)    
        
    print(model_name, feature.shape)
    np.save(model_save_fold + 'data.npy', feature)
    np.save(model_save_fold + 'label.npy', label)



(65, 52, 125)
(65, 52, 125)
(65, 52, 125)
(65, 52, 125)
chao_cfnn (65, 26000)
li_svm (65, 520)


/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/utils/fnirs_utils.py:176: RuntimeWarning: invalid value encountered in true_divide
  normalized_data[i, :] = (data[i, :] - mean) / std
/home/jy/miniconda3/envs/tf/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4424: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


duan_rsfc (65, 52, 52)


/home/jy/miniconda3/envs/tf/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 125, using nperseg = 125
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/jy/miniconda3/envs/tf/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 4 is too high: all coefficients will experience boundary effects.
  warnings.warn(


wang_alex (65, 52, 66)
yu_gnn (65, 52, 36)


/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/utils/fnirs_utils.py:485: RuntimeWarning: divide by zero encountered in true_divide
  feature_skewness /= np.std(Hb, axis=2) ** 3
/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/utils/fnirs_utils.py:497: RuntimeWarning: invalid value encountered in true_divide
  feature_kurtosis = (n * (n + 1) * fourth_moment) / ((n - 1) * (n - 2)
/home/jy/miniconda3/envs/tf/lib/python3.9/site-packages/numpy/core/_methods.py:232: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)
/home/jy/Documents/fnirs/treatment_response/fnirs-depression-deeplearning/utils/fnirs_utils.py:176: RuntimeWarning: invalid value encountered in subtract
  normalized_data[i, :] = (data[i, :] - mean) / std


zhu_xgboost (65, 520)


In [17]:
hbo_data, hbr_data = get_avg_hbo_hbr()
model_save_fold = MAIN_FOLD + 'yu_gnn' + '/'
print(np.max(hbo_data), np.min(hbo_data), np.max(hbr_data), np.min(hbr_data))

19.174756038255552 -15.64890383615652 19.550877698071083 -15.845824740709954


In [14]:

from joblib import Parallel, delayed
from scipy import signal


# 假设 hbo_data 和 hbr_data 是 numpy 数组，形状为 (100, 52, 125)
fs = 1  # 设置您的采样频率
# 定义一个函数来计算两个信号之间的 CSD

def compute_csd(x, y, fs):
    f, Pxy = signal.csd(x, y, fs)
    return Pxy.mean()


# 对于每个主题并行计算 CSD
hbo_coh = Parallel(n_jobs=-1)(
    delayed(compute_csd)(hbo_data[i, j], hbo_data[i, k], fs)
    for i in range(hbo_data.shape[0])
    for j in range(hbo_data.shape[1])
    for k in range(hbo_data.shape[1])
)


hbr_coh = Parallel(n_jobs=-1)(
    delayed(compute_csd)(hbr_data[i, j], hbr_data[i, k], fs)
    for i in range(hbr_data.shape[0])
    for j in range(hbr_data.shape[1])
    for k in range(hbr_data.shape[1])
)


spatial_hbo_coh_output = np.array(hbo_coh).reshape(hbo_data.shape[0],hbo_data.shape[1],hbo_data.shape[1])
spatial_hbr_coh_output = np.array(hbr_coh).reshape(hbr_data.shape[0],hbr_data.shape[1],hbr_data.shape[1])

# Define a function to compute the Pearson's correlation coefficient
def compute_correlation(x, y):
    corr, _ = stats.pearsonr(x, y)
    return corr


# Compute correlation for hbo data
hbo_cor = Parallel(n_jobs=-1)(
    delayed(compute_correlation)(hbo_data[i, j], hbo_data[i, k])
    for i in range(hbo_data.shape[0])
    for j in range(52)
    for k in range(52)
)

# Compute correlation for hbr data
hbr_cor = Parallel(n_jobs=-1)(
    delayed(compute_correlation)(hbr_data[i, j], hbr_data[i, k])
    for i in range(hbr_data.shape[0])
    for j in range(52)
    for k in range(52)
)


spatial_hbo_cor_output = np.array(hbo_cor).reshape(hbo_data.shape[0],hbo_data.shape[1],hbo_data.shape[1])
spatial_hbr_cor_output = np.array(hbr_cor).reshape(hbo_data.shape[0],hbo_data.shape[1],hbo_data.shape[1])


In [16]:
def clear_Nan(A):
    xA = A 
    xA[np.isinf(xA)] = np.nan
    no_nan_A =  np.nan_to_num(xA, copy=False)
    return no_nan_A

nor_spatial_hbo_coh_output = np.abs(clear_Nan(spatial_hbo_coh_output))
nor_spatial_hbo_cor_output = np.abs(clear_Nan(spatial_hbo_cor_output))
nor_spatial_hbr_coh_output = np.abs(clear_Nan(spatial_hbr_coh_output))
nor_spatial_hbr_cor_output = np.abs(clear_Nan(spatial_hbr_cor_output))
A = np.concatenate((nor_spatial_hbo_coh_output[...,np.newaxis],
                    nor_spatial_hbo_cor_output[...,np.newaxis],
                    nor_spatial_hbr_coh_output[...,np.newaxis],
                    nor_spatial_hbr_cor_output[...,np.newaxis]),
                   axis=3)
print(A.shape)

np.save(model_save_fold + 'adj.npy', A)
np.save(model_save_fold + 'adj_1.npy', A[..., 1])

(65, 52, 52, 4)


In [18]:
def normalize(data):
    # Iterate over each subject
    normalized_data = np.empty_like(data)
    for i in range(data.shape[0]):
        # Calculate the mean and standard deviation for the current subject
        mean = np.mean(data[i])
        std = np.std(data[i])

        # Perform z-normalization for the current subject
        normalized_data[i] = (data[i] - mean) / std
    return normalized_data
def clear_Nan(A):
    xA = A 
    xA[np.isinf(xA)] = np.nan
    no_nan_A =  np.nan_to_num(xA, copy=False)
    return no_nan_A

nor_spatial_hbo_coh_output = np.abs(normalize(clear_Nan(spatial_hbo_coh_output)))
nor_spatial_hbo_cor_output = np.abs(normalize(clear_Nan(spatial_hbo_cor_output)))
nor_spatial_hbr_coh_output = np.abs(normalize(clear_Nan(spatial_hbr_coh_output)))
nor_spatial_hbr_cor_output = np.abs(normalize(clear_Nan(spatial_hbr_cor_output)))
A = np.concatenate((nor_spatial_hbo_coh_output[...,np.newaxis],
                    nor_spatial_hbo_cor_output[...,np.newaxis],
                    nor_spatial_hbr_coh_output[...,np.newaxis],
                    nor_spatial_hbr_cor_output[...,np.newaxis]),
                   axis=3)
print(A.shape)

np.save(model_save_fold + 'normalized_adj.npy', A)
np.save(model_save_fold + 'normalized_adj_1.npy', A[..., 1])


(65, 52, 52, 4)
